# Preprocessing House Prices - Advanced Regression Techniques

### Import thư viện

In [96]:
import pandas as pd
import numpy as np
import scipy.stats
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
import optuna

train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')


In [97]:
train_df.shape

(1460, 81)

## Preprocessing

### Remove Outliers

In [98]:
# train_clean = train_df.copy()
# outlier_cols = ['GrLivArea', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF']
# for col in outlier_cols:
#     if col in train_clean.columns:
#         Q1 = train_clean[col].quantile(0.25)
#         Q3 = train_clean[col].quantile(0.75)
#         IQR = Q3 - Q1
#         lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
#         train_clean = train_clean[(train_clean[col] >= lower) & (train_clean[col] <= upper)]
    
# print(f"Loại outlier: {len(train_df) - len(train_clean)} mẫu outlier")

# train_df = train_clean.reset_index(drop=True)
# train_df.shape

### Merge the datasets

In [99]:
df = pd.concat([train_df, test_df], axis=0, sort=False)

df.shape

(2919, 81)

### Fill missing value

In [100]:
def missing_values(df):
    missing_values = df.isnull().sum()
    missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
    return missing_values
print(missing_values(df))

PoolQC          2909
MiscFeature     2814
Alley           2721
Fence           2348
MasVnrType      1766
SalePrice       1459
FireplaceQu     1420
LotFrontage      486
GarageCond       159
GarageYrBlt      159
GarageFinish     159
GarageQual       159
GarageType       157
BsmtExposure      82
BsmtCond          82
BsmtQual          81
BsmtFinType2      80
BsmtFinType1      79
MasVnrArea        23
MSZoning           4
Functional         2
BsmtHalfBath       2
BsmtFullBath       2
Utilities          2
KitchenQual        1
TotalBsmtSF        1
BsmtUnfSF          1
GarageCars         1
GarageArea         1
BsmtFinSF2         1
BsmtFinSF1         1
Exterior2nd        1
Exterior1st        1
SaleType           1
Electrical         1
dtype: int64


In [101]:
none_cols = [
    'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'MasVnrType', 'FireplaceQu',
    'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
    'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
    'MasVnrType', 'PoolQC'
    ]
for col in none_cols:
    df[col] = df[col].fillna('None')

df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0)
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)
df['LotFrontage'] = df['LotFrontage'].fillna(0)

cols = [
    'Electrical', 'KitchenQual', 'Exterior1st', 'Exterior2nd',
    'SaleType', 'MSZoning', 'Functional', 'Utilities',
    'BsmtFullBath', 'BsmtHalfBath', 'GarageCars', 'GarageArea',
    'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF'
]
for col in cols:
    df[col] = df[col].fillna(df[col].mode()[0])

print(missing_values(df))

SalePrice    1459
dtype: int64


### Features engineering

In [102]:
# Các tạo các đặc trưng mới về tuổi của ngôi nhà

df['HouseAge'] = df['YrSold'] - df['YearBuilt']
df['RemodAge'] = df['YrSold'] - df['YearRemodAdd']
df['GarageAge'] = df['YrSold'] - df['GarageYrBlt']

df.drop(['YrSold', 'MoSold'], inplace=True, axis=1)


In [103]:
# Tạo các đặc trưng tổng diện tích sử dụng
df['BsmtFinSF'] = df['BsmtFinSF1'] + df['BsmtFinSF2']

df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']

df.drop(['1stFlrSF', '2ndFlrSF', 'BsmtFinSF1', 'BsmtFinSF2'], axis=1, inplace=True)

In [104]:
# Các đặc trưng liên quan đến phòng tắm
df['TotalBath'] = (df['FullBath'] + (0.5 * df['HalfBath']) +
                   df['BsmtFullBath'] + (0.5 * df['BsmtHalfBath']))

df.drop(['FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath'], axis=1, inplace=True)

In [105]:
# Các đặc trưng liên quan đến Garage
df['GarageAreaPerCar'] = df['GarageArea'] / df['GarageCars']

df['GarageAreaPerCar'] = df['GarageAreaPerCar'].fillna(0)


In [106]:
# Các đặc trưng nhị phân thể hiện sự tồn tại của các tiện ích
df['HasPool'] = (df['PoolArea'] > 0).astype(int)
df['HasGarage'] = (df['GarageArea'] > 0).astype(int)
df['HasFireplace'] = (df['Fireplaces'] > 0).astype(int)
df['HasBsmt'] = (df['TotalBsmtSF'] > 0).astype(int)

df.drop(['GarageArea', 'GarageCars'], axis=1, inplace=True)

### Data tranformation

In [107]:
numerical_cols = [ cname for cname in df.columns if df[cname].dtypes!= 'object' and cname != 'SalePrice' ]

skew_df = pd.DataFrame(numerical_cols, columns=['Feature'])
skew_df['Skew'] = skew_df['Feature'].apply(lambda feature: scipy.stats.skew(df[feature]))
skew_df['Absolute Skew'] = skew_df['Skew'].abs()
skew_df['Skew'] = skew_df['Absolute Skew'].apply(lambda x: True if x >= 0.5 else False)

skew_df

,Feature,Skew,Absolute Skew
0,Id,False,0.000000
1,MSSubClass,True,1.375457
2,LotFrontage,False,0.022013
3,LotArea,True,12.822431
4,OverallQual,False,0.197110
5,OverallCond,True,0.570312
6,YearBuilt,True,0.599806
7,YearRemodAdd,False,0.451020
8,MasVnrArea,True,2.613592
9,BsmtUnfSF,True,0.919339


In [108]:
for col in skew_df[skew_df['Skew'] == True]['Feature']:
    df[col] = np.log1p(df[col])

c:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


### Encoding feature

In [109]:
categorical_cols = df.select_dtypes(include=['object']).columns

df_categorical = df[categorical_cols].copy()

In [110]:
ordinal_1 = ['ExterQual' , 'ExterCond' , 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']

for col in ordinal_1:
    if 'None' in df[col].value_counts().index:
        df[col] = df[col].map({'None':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}).astype(int)
    else:
        df[col] = df[col].map({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}).astype(int)

In [111]:
ordinal_2 = ['LotShape','LandContour','Utilities' ,'CentralAir','PavedDrive', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Functional', 'GarageFinish', 'Fence', 'Heating', 'LandSlope']

df['LotShape'] = df['LotShape'].map({'None':0, 'IR3':1, 'IR2':2, 'IR1':3, 'Reg':4}).astype(int)
df['LandContour'] = df['LandContour'].map({"None":0, "Low":1, "HLS":2, "Bnk":3, "Lvl":4}).astype(int)
df['Utilities'] = df['Utilities'].map({"None":0, "ELO":1, "NoSeWa":2, "NoSewr":3, "AllPub":4}).astype(int)
df['CentralAir'] = df['CentralAir'].map({'None':0, 'N':1 ,'Y':2,}).astype(int)
df['PavedDrive'] = df['PavedDrive'].map({'None':0, 'N':1,'P':2,'Y':3}).astype(int)
df['BsmtExposure'] = df['BsmtExposure'].map({'None':0, 'No':1,'Mn':2,'Av':3,'Gd':4}).astype(int)
df['BsmtFinType1'] = df['BsmtFinType1'].map({'None':0, 'Unf':1,'LwQ':2,'Rec':3,'BLQ':4,'ALQ':5,'GLQ':6}).astype(int)
df['BsmtFinType2'] = df['BsmtFinType2'].map({'None':0, 'Unf':1,'LwQ':2,'Rec':3,'BLQ':4,'ALQ':5,'GLQ':6}).astype(int)
df['Functional'] = df['Functional'].map({'None':0, 'Sal':1,'Sev':2,'Maj2':3,'Maj1':4,'Mod':5,'Min2':6,'Min1':7,'Typ':8}).astype(int)
df['GarageFinish'] = df['GarageFinish'].map({'None':0, 'Unf':1,'RFn':2,'Fin':3}).astype(int)
df['Fence'] = df['Fence'].map({'None':0, 'MnWw':1,'GdWo':2,'MnPrv':3,'GdPrv':4}).astype(int)
df['Heating'] = df['Heating'].map({'None':0, 'Floor':1,'Wall':2,'OthW':3,'GasA':4,'GasW':5,'Grav':6,'Water':7,'Steam':8}).astype(int)
df['LandSlope'] = df['LandSlope'].map({'None':0, 'Sev':1,'Mod':2,'Gtl':3}).astype(int)


In [112]:
# tách các cột non-ordinal còn lại từ df_categorical

df_categorical = df_categorical.drop(columns = ordinal_1) 
df_categorical = df_categorical.drop(columns = ordinal_2) 

ohe = df_categorical.columns.tolist()

df_encoded = pd.get_dummies(df_categorical, columns=ohe, drop_first=True).astype(int)

df = pd.concat([df.drop(columns = ohe), df_encoded], axis=1)


### Split train, test data

In [113]:
df_train = df.iloc[:train_df.shape[0]]

df_test = df.iloc[train_df.shape[0]:]

df_train.drop(['Id'], inplace=True, axis=1)
df_test.drop(['Id', 'SalePrice'], inplace=True, axis=1)

C:\Users\MSI VN\AppData\Local\Temp\ipykernel_9772\1989219299.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(['Id'], inplace=True, axis=1)
C:\Users\MSI VN\AppData\Local\Temp\ipykernel_9772\1989219299.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop(['Id', 'SalePrice'], inplace=True, axis=1)


### Encode SalePrice

In [114]:
log_target = np.log1p(df_train['SalePrice'])
df_train.drop('SalePrice', axis=1, inplace=True)

C:\Users\MSI VN\AppData\Local\Temp\ipykernel_9772\556502089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop('SalePrice', axis=1, inplace=True)


In [115]:
# def preprocess_data_fixed(train_df, test_df):
#     # ================================
#     # 1. LOẠI OUTLIER TRÊN TRAIN CHỈ SAU KHI TÁCH
#     # ================================
#     train_clean = train_df.copy()
#     outlier_cols = ['GrLivArea', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF']
#     for col in outlier_cols:
#         if col in train_clean.columns:
#             Q1 = train_clean[col].quantile(0.25)
#             Q3 = train_clean[col].quantile(0.75)
#             IQR = Q3 - Q1
#             lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
#             train_clean = train_clean[(train_clean[col] >= lower) & (train_clean[col] <= upper)]
    
#     print(f"Loại outlier: {len(train_df) - len(train_clean)} mẫu")

#     # ================================
#     # 2. CHỈ DÙNG TRAIN ĐỂ XÂY DỰNG FEATURES
#     # ================================
#     X_train = train_clean.drop('SalePrice', axis=1)
#     y_train = np.log1p(train_clean['SalePrice'])  # log ngay
#     X_test = test_df.copy()

#     # Feature engineering (chỉ dùng train để tính mean/mode)
#     def add_features(df, is_train=True):
#         df = df.copy()
#         df['TotalSF'] = df['TotalBsmtSF'].fillna(0) + df['1stFlrSF'] + df['2ndFlrSF'].fillna(0)
#         df['TotalBath'] = (df['FullBath'] + 0.5*df['HalfBath'] + 
#                            df['BsmtFullBath'].fillna(0) + 0.5*df['BsmtHalfBath'].fillna(0))
#         df['Age'] = df['YrSold'] - df['YearBuilt']
#         df['GarageAge'] = df['YrSold'] - df['GarageYrBlt'].fillna(df['YrSold'])
#         df['RemodAge'] = df['YrSold'] - df['YearRemodAdd']
#         df['HasPool'] = (df['PoolArea'] > 0).astype(int)
#         df['HasGarage'] = (df['GarageArea'] > 0).astype(int)
#         df['HasFireplace'] = (df['Fireplaces'] > 0).astype(int)
#         df['HasBsmt'] = (df['TotalBsmtSF'] > 0).astype(int)
#         return df

#     X_train = add_features(X_train, is_train=True)
#     X_test = add_features(X_test, is_train=False)

#     cols_to_drop = [
#     'Id',
#     'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
#     'FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath',
#     'YearBuilt','YearRemodAdd',
#     'GarageArea',
#     'Id', 'YrSold', 'MoSold', 'MiscVal', 'LowQualFinSF',
#     'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'GarageCars'
#     ]

#     X_train = X_train.drop(columns=[c for c in cols_to_drop if c in X_train.columns], errors='ignore')
#     X_test = X_test.drop(columns=[c for c in cols_to_drop if c in X_test.columns], errors='ignore')

#     # ================================
#     # 3. FILL MISSING – CHỈ DỰA VÀO TRAIN
#     # ================================
#     none_cols = [
#         'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'MasVnrType', 'FireplaceQu',
#         'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
#         'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
#         'MasVnrType'
#     ]
#     for col in none_cols:
#         if col in X_train.columns:
#             X_train[col] = X_train[col].fillna('None')
#             X_test[col] = X_test[col].fillna('None')

#     X_train['GarageYrBlt'] = pd.to_numeric(X_train['GarageYrBlt'], errors='coerce').fillna(0).astype(int)
#     X_train['MasVnrArea'] = X_train['MasVnrArea'].fillna(0)
#     X_train['LotFrontage'] = X_train['LotFrontage'].fillna(0)

#     X_test['GarageYrBlt'] = pd.to_numeric(X_test['GarageYrBlt'], errors='coerce').fillna(0).astype(int)
#     X_test['MasVnrArea'] = X_test['MasVnrArea'].fillna(0)
#     X_test['LotFrontage'] = X_test['LotFrontage'].fillna(0)


#     mode_cols = ['Electrical', 'KitchenQual', 'Exterior1st', 'Exterior2nd',
#                  'SaleType', 'MSZoning', 'Functional', 'Utilities']
#     for col in mode_cols:
#         if col in X_train.columns:
#             mode_val = X_train[col].mode()[0]
#             X_train[col] = X_train[col].fillna(mode_val)
#             X_test[col] = X_test[col].fillna(mode_val)

#     # ================================
#     # 4. ORDINAL ENCODING (riêng)
#     # ================================
#     ordinal_map = {
#         'ExterQual': {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
#         'ExterCond': {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
#         'BsmtQual': {'None':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
#         'HeatingQC': {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
#         'KitchenQual': {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
#         'FireplaceQu': {'None':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
#         'GarageQual': {'None':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
#         'GarageCond': {'None':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
#         'PoolQC': {'None':0, 'Fa':1, 'TA':2, 'Gd':3, 'Ex':4},
#         'BsmtExposure': {'None':0, 'No':1, 'Mn':2, 'Av':3, 'Gd':4},
#         'BsmtFinType1': {'None':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6},
#         'GarageFinish': {'None':0, 'Unf':1, 'RFn':2, 'Fin':3},
#         'PavedDrive': {'N':1, 'P':2, 'Y':3},
#         'CentralAir': {'N':0, 'Y':1},
#         'Functional': {'Sal':1, 'Sev':2, 'Maj2':3, 'Maj1':4, 'Mod':5, 'Min2':6, 'Min1':7, 'Typ':8}
#     }

#     for col, mapping in ordinal_map.items():
#         if col in X_train.columns:
#             X_train[col] = X_train[col].map(mapping).fillna(0).astype(int)
#             X_test[col] = X_test[col].map(mapping).fillna(0).astype(int)

#     # ================================
#     # 5. ONE-HOT ENCODING – CHỈ TRAIN
#     # ================================
#     cat_cols = X_train.select_dtypes(include='object').columns
#     X_train_encoded = pd.get_dummies(X_train, columns=cat_cols, drop_first=True)
#     X_test_encoded = pd.get_dummies(X_test, columns=cat_cols, drop_first=True)

#     # Align columns
#     X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)
#     X_test_encoded = X_test_encoded.drop('SalePrice', axis=1, errors='ignore')

#     # ================================
#     # 6. LOG SKEW – CHỈ TRAIN
#     # ================================
#     num_cols = X_train_encoded.select_dtypes(include=['float64', 'int64']).columns
#     skew_feats = X_train_encoded[num_cols].apply(lambda x: scipy.stats.skew(x.dropna()))
#     skew_feats = skew_feats[abs(skew_feats) > 0.75].index

#     for feat in skew_feats:
#         if feat != 'SalePrice':
#             X_train_encoded[feat] = np.log1p(X_train_encoded[feat])
#             X_test_encoded[feat] = np.log1p(X_test_encoded[feat])

#     return X_train_encoded, y_train, X_test_encoded

## train model

In [116]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error



In [117]:
X = df_train.copy()
y = log_target.copy()  # log ngay

X_tr, X_val, y_tr, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=None
)

# 4. Các model
models = {
    'Ridge'        : Ridge(alpha=1.0),
    'RF'           : RandomForestRegressor(n_estimators=200, random_state=42),
    'GBR'          : GradientBoostingRegressor(n_estimators=200, random_state=42),
    'XGBoost'      : XGBRegressor(n_estimators=300, random_state=42, verbosity=0),
    'LightGBM'     : LGBMRegressor(n_estimators=300, random_state=42, verbose=-1),
    'CatBoost'     : CatBoostRegressor(n_estimators=300, random_state=42, verbose=0),
}

# 5. Đánh giá
results = []
print("\n=== Đánh giá nhanh ===")
for name, model in models.items():
    model.fit(X_tr, y_tr)
    pred = model.predict(X_val)               # ← X_val, không phải y_val
    rmse = np.sqrt(mean_squared_error(y_val, pred))
    results.append({'Model': name, 'RMSE': rmse})
    print(f"{name:12} → {rmse:.5f}")

res_df = pd.DataFrame(results).sort_values('RMSE').reset_index(drop=True)
print("\n=== TOP 3 ===")
print(res_df.head(3))


=== Đánh giá nhanh ===
Ridge        → 0.12573
RF           → 0.14839
GBR          → 0.13847
XGBoost      → 0.14381
LightGBM     → 0.14485
CatBoost     → 0.13414

=== TOP 3 ===
      Model      RMSE
0     Ridge  0.125725
1  CatBoost  0.134138
2       GBR  0.138467


## Hyperparamter Tuning

In [ ]:
def objective(trial):
    model_name = trial.suggest_categorical('model', ['Ridge', 'CatBoost', 'GBR'])
    
    if model_name == 'Ridge':
        params = {
            'alpha': trial.suggest_float('alpha', 0.01, 100, log=True),
            'solver': trial.suggest_categorical('solver', ['svd', 'cholesky', 'lsqr']),
            'max_iter': 10000
        }
        model = Ridge(**params)
    
    elif model_name == 'CatBoost':
        params = {
            'iterations'    : trial.suggest_int('iterations', 500, 1500),
            'learning_rate' : trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
            'depth'         : trial.suggest_int('depth', 6, 10),
            'l2_leaf_reg'   : trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
            'border_count'  : trial.suggest_int('border_count', 128, 254),
            'random_state'  : 42,
            'verbose'       : 0
        }
        model = CatBoostRegressor(**params)
    
    # else:  # RF
    #     params = {
    #         'n_estimators'      : trial.suggest_int('n_estimators', 500, 1500),
    #         'max_depth'         : trial.suggest_int('max_depth', 10, 30),
    #         'min_samples_split' : trial.suggest_int('min_samples_split', 2, 10),
    #         'min_samples_leaf'  : trial.suggest_int('min_samples_leaf', 1, 5),
    #         'max_features'      : trial.suggest_categorical('max_features', ['sqrt', 'log2', 0.8]),
    #         'random_state'      : 42
    #     }
    #     model = RandomForestRegressor(**params)
    else: # GRB
        params = {
            'n_estimators'      : trial.suggest_int('n_estimators', 500, 1500),
            'learning_rate'     : trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
            'max_depth'         : trial.suggest_int('max_depth', 3, 10),
            'subsample'         : trial.suggest_float('subsample', 0.6, 1.0),
            'min_samples_split' : trial.suggest_int('min_samples_split', 2, 10),
            'min_samples_leaf'  : trial.suggest_int('min_samples_leaf', 1, 5),
            'max_features'      : trial.suggest_categorical('max_features', ['sqrt', 'log2', 0.8]),
            'random_state'      : 42
        }
        model = GradientBoostingRegressor(**params)

    # 5-fold CV
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmses = []
    for tr_idx, val_idx in kf.split(X):
        model.fit(X.iloc[tr_idx], y.iloc[tr_idx])
        pred = model.predict(X.iloc[val_idx])
        rmses.append(np.sqrt(mean_squared_error(y.iloc[val_idx], pred)))
    return np.mean(rmses)

# CHẠY
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)
print(study.best_params)
print(f"Best RMSE: {study.best_value:.5f}")


[I 2025-11-11 21:53:12,398] A new study created in memory with name: no-name-5f087991-b603-4a88-b974-e850857c0f6a
[I 2025-11-11 21:53:12,488] Trial 0 finished with value: 0.13885529322707252 and parameters: {'model': 'Ridge', 'alpha': 47.775707260295796, 'solver': 'cholesky'}. Best is trial 0 with value: 0.13885529322707252.
[I 2025-11-11 21:53:12,717] Trial 1 finished with value: 0.13820061955552462 and parameters: {'model': 'Ridge', 'alpha': 0.08411499027536601, 'solver': 'svd'}. Best is trial 1 with value: 0.13820061955552462.
[I 2025-11-11 21:53:12,915] Trial 2 finished with value: 0.13295924682595311 and parameters: {'model': 'Ridge', 'alpha': 1.4047693097232357, 'solver': 'svd'}. Best is trial 2 with value: 0.13295924682595311.
[I 2025-11-11 21:53:45,364] Trial 3 finished with value: 0.12715547458548235 and parameters: {'model': 'CatBoost', 'iterations': 832, 'learning_rate': 0.06653273099080237, 'depth': 7, 'l2_leaf_reg': 5.0159590128765785, 'border_count': 135}. Best is trial 3

{'model': 'CatBoost', 'iterations': 1408, 'learning_rate': 0.03223120791573973, 'depth': 8, 'l2_leaf_reg': 9.99876931885033, 'border_count': 250}
Best RMSE: 0.12610


In [136]:
# Lấy toàn bộ kết quả từ Optuna
results_df = study.trials_dataframe()

# Tạo dict chứa best params cho từng model
best_params_per_model = {}

# Duyệt qua từng model name có trong trials
for model_name in results_df['params_model'].unique():
    # Lọc trials của model đó
    sub_df = results_df[results_df['params_model'] == model_name]
    
    # Lấy trial có giá trị 'value' (RMSE) nhỏ nhất
    best_trial = sub_df.loc[sub_df['value'].idxmin()]
    
    # Lấy các tham số thuộc model đó
    params = {
        col.replace('params_', ''): best_trial[col]
        for col in results_df.columns
        if col.startswith('params_')
    }
    
    # Xóa param không thuộc model này
    valid_params = {}
    if model_name == 'Ridge':
        allow = ['alpha', 'solver', 'max_iter']
    elif model_name == 'CatBoost':
        allow = ['iterations', 'learning_rate', 'depth', 'l2_leaf_reg', 'border_count', 'random_state', 'verbose']
    elif model_name == 'GBR':
        allow = ['n_estimators', 'learning_rate', 'max_depth', 'subsample', 'min_samples_split', 'min_samples_leaf', 'max_features', 'random_state']
    else:
        allow = []
    
    for k in params:
        if k in allow:
            val = params[k]
            # Ép kiểu cho các tham số cần là int
            if k in ['max_depth', 'n_estimators', 'min_samples_split', 'min_samples_leaf', 
                    'iterations', 'depth', 'border_count']:
                val = int(val)
            valid_params[k] = val

    
    best_params_per_model[model_name] = valid_params

# Hiển thị 3 bộ tham số tốt nhất
from pprint import pprint
pprint(best_params_per_model)


{'CatBoost': {'border_count': 250,
              'depth': 8,
              'iterations': 1408,
              'l2_leaf_reg': 9.99876931885033,
              'learning_rate': 0.03223120791573973},
 'GBR': {'learning_rate': 0.03331521271771233,
         'max_depth': 5,
         'max_features': 'sqrt',
         'min_samples_leaf': 4,
         'min_samples_split': 2,
         'n_estimators': 1000,
         'subsample': 0.8644989030857276},
 'Ridge': {'alpha': 1.4047693097232357, 'solver': 'svd'}}


In [139]:
ridge_best = Ridge(**best_params_per_model.get('Ridge', {}))
cat_best   = CatBoostRegressor(**best_params_per_model.get('CatBoost', {}), verbose=0)
gbr_best   = GradientBoostingRegressor(**best_params_per_model.get('GBR', {}))


# 3. Huấn luyện trên toàn bộ dữ liệu
print("\nHuấn luyện model trên toàn bộ train...")
ridge_best.fit(X, y)
cat_best.fit(X, y)
gbr_best.fit(X, y)

# 4. Dự đoán trên test
print("Dự đoán trên test set...")
df_test.replace([np.inf, -np.inf], np.nan, inplace=True)
df_test.fillna(0, inplace=True)
ridge_pred = np.expm1(ridge_best.predict(df_test))
cat_pred   = np.expm1(cat_best.predict(df_test))
gbr_pred   = np.expm1(gbr_best.predict(df_test))

final_preds = 0.4 * ridge_pred + 0.5 * cat_pred + 0.1 * gbr_pred

# 5. Tạo submission
submission = pd.DataFrame({
    'Id': test_df['Id'].astype(int),
    'SalePrice': final_preds
})

# 6. Lưu file
submission.to_csv('./submission/submission.csv', index=False)

print(f"\nSUBMISSION ĐÃ SẴN SÀNG!")
print(f"   Số mẫu: {len(submission)}")
print(f"   Giá min: {final_preds.min():.0f}, max: {final_preds.max():.0f}")


Huấn luyện model trên toàn bộ train...
Dự đoán trên test set...

SUBMISSION ĐÃ SẴN SÀNG!
   Số mẫu: 1459
   Giá min: 47589, max: 493334


## Experirment

1. 15:54, 11/11/2025 lần test đầu tiên,ridge là cao nhất, kaggle 0.13422
2. 16:07, 11/11/2025 thêm 2 feature 'TotalPorchSF', 'LivLotRatio', rigde là model tốt nhất, kaggle 0.13439
3. 16:25, 11/11/2025 log tranform cho 2 feature mới, drop các cột dùng để tạo feature, kaggle tăng 0.14 => ko nên thêm 2 ffeature
4. 10:09, 11/11/2025 preprocess data cách khác, kaggle giảm còn 0.12175
5. 10:47, 11/11/2025 weighted blending, kaggle giảm còn 0.11855

Mai nhớ thử remove outlier